# Sentiment Analysis con KNN
## Clasificador en C++ 💪💪
Vamos a probar a nuestro bichito


Definir los path al ejecutable de python 3.6 y sus librerías,
de acuerdo al virtual env que estén corriendo.

In [1]:
!cd .. && git submodule init
!cd .. && git submodule update
!cd .. && mkdir build
!cd ../build/ && rm -rf *
!cd ../build && cmake \
  -DPYTHON_EXECUTABLE="$(which python)" \
  -DCMAKE_BUILD_TYPE=Release ..
!cd ../build && make install

mkdir: cannot create directory ‘build’: File exists
-- The C compiler identification is GNU 7.4.0
-- The CXX compiler identification is GNU 7.4.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
Release mode
-- Found PythonInterp: /mnt/c/users/ami/desktop/Facultad/Métodos/tp2/metnum-tp2-20192c/tp2/bin/python (found version "3.6.8") 
-- Found PythonLibs: /usr/lib/x86_64-linux-gnu/libpython3.6m.so
-- pybind11 v2.3.dev0
-- Performing Test HAS_FLTO
-- Performing Test HAS_FLTO - Success
-- LTO enabled
CMAKE_INSTALL_PREFIX=/mnt/c/users/ami/desktop/F

In [4]:
# Verifico la correcta instalación. Si no falla el import está OK
!pwd
!python --version
import sentiment


/mnt/c/users/ami/desktop/Facultad/Métodos/tp2/metnum-tp2-20192c/notebooks
Python 3.6.8


In [5]:
%load_ext autoreload
%autoreload 2
import pandas as pd

!cd ../data && tar -xvf *.tgz
!cd ../data && tar -xvf *.tar.gz

df = pd.read_csv("../data/imdb_small.csv", index_col=0)

print("Cantidad de documentos: {}".format(df.shape[0]))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
tar: *.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now
tar: *.tar.gz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now
Cantidad de documentos: 12500


In [6]:
df.describe()

,type,review,label,file
count,12500,12500,12500,12500
unique,2,12469,2,12085
top,test,"Seriously, I donÂ´t really get why people here...",neg,10628_1.txt
freq,6275,2,6322,2


In [7]:
text_train = (df[df.type == 'train']["review"])[:1000]
label_train = (df[df.type == 'train']["label"])[:1000]

text_test = (df[df.type == 'test']["review"])[:500]
label_test = (df[df.type == 'test']["label"])[:500]

print("Cantidad de instancias de entrenamiento = {}".format(len(text_train)))
print("Cantidad de instancias de test = {}".format(len(text_test)))


Cantidad de instancias de entrenamiento = 1000
Cantidad de instancias de test = 500


In [8]:
print("Class balance : {} pos {} neg".format(
    (label_train == 'pos').sum() / label_train.shape[0], 
    (label_train == 'neg').sum() / label_train.shape[0]
))

Class balance : 0.512 pos 0.488 neg


In [9]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_df=0.90, min_df=0.01, max_features=5000)

vectorizer.fit(text_train)

X_train, y_train = vectorizer.transform(text_train), (label_train == 'pos').values
X_test, y_test = vectorizer.transform(text_test), (label_test == 'pos').values

In [10]:
import sentiment

clf = sentiment.KNNClassifier(100)

clf.fit(X_train, y_train)


In [11]:
%%time
from sklearn.metrics import accuracy_score
y_pred = clf.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print("Accuracy: {}".format(acc))

Accuracy: 0.572
CPU times: user 14.2 s, sys: 29.4 s, total: 43.6 s
Wall time: 47.6 s


In [25]:
import matplotlib.pyplot as plt
import sentiment
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
df = pd.read_csv("../data/imdb_small.csv", index_col=0)

#codigo de countvectorizer
#randomizar la base
df = df.sample(frac=1)


        type                                             review label  \
id                                                                      
19631   test  This movie is my families favorite Christmas m...   pos   
32878  train  Spoiler below, but read on or you'll never kno...   neg   
28509  train  One of the worst romantic comedies (nay, worst...   neg   
32451  train  An actor asks, "What's my motivation?," to und...   neg   
45002  train  Whether this movie is propaganda or not (I fir...   pos   
39719  train  I have seen this wonderful production, and I w...   pos   
38179  train  What another reviewer called lack of character...   pos   
38422  train  I enjoyed Still Crazy more than any film I hav...   pos   
21771   test  What impressed me the most about "One True Thi...   pos   
9928    test  This film is unbelievable on any level. It fai...   neg   

               file  
id                   
19631   5169_10.txt  
32878    5841_1.txt  
28509    1909_1.txt  
32451    5457

In [ ]:
# TEST DE CAMBIO DE N_NEIGHBORS (SOLO KNN)
# ======== OBSOLETO =========
#Temporal cambiar 100 a 6225 y sacar limite superior test
text_train = df[:6225]["review"]
label_train = df[:6225]["label"]

text_test = df[6225:]["review"]
label_test = df[6225:]["label"]

print("Cantidad de instancias de entrenamiento = {}".format(len(text_train)))
print("Cantidad de instancias de test = {}".format(len(text_test)))

vectorizer = CountVectorizer(max_df=0.90, min_df=0.01, max_features=5000)
vectorizer.fit(text_train)
X_train, y_train = vectorizer.transform(text_train), (label_train == 'pos').values
X_test, y_test = vectorizer.transform(text_test), (label_test == 'pos').values

acc = []
neighs = list(range(25,201,25))
for n in neighs:
    clf = sentiment.KNNClassifier(n)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    acc.append(accuracy_score(y_test, y_pred))
    print("Accuracy {}: {}".format(n,acc))
    
plt.plot(neighs, acc, 'g-o')
plt.xlabel("Vecinos")
plt.ylabel("Accuracy")
plt.xticks(neighs)
plt.title("Performance de clasificador en función a cantidad de vecinos");

Cantidad de instancias de entrenamiento = 6225
Cantidad de instancias de test = 6275


In [1]:
# TEST DE CAMBIO DE N_NEIGHBORS Y ALPHA 
# (es decir, el de Rodri)

#SACAR ESTOS IMPORTS Y HACER Q ANDEN CON LOS DE ARRIBA
import pandas as pd
from sklearn.metrics import accuracy_score
df = pd.read_csv("../data/imdb_small.csv", index_col=0)

from sklearn.feature_extraction.text import CountVectorizer
from sentiment import PCA, KNNClassifier, get_first_eigenvalues
import numpy as np
import scipy
import matplotlib.pyplot as plt

# CAMBIAR 6225 POR 3125 (AMI) Y 9375 (JULI)
borde = 9375
text_train = df[:borde]["review"]
label_train = df[:borde]["label"]

text_test = df[borde:]["review"]
label_test = df[borde:]["label"]

print("Cantidad de instancias de entrenamiento = {}".format(len(text_train)))
print("Cantidad de instancias de test = {}".format(len(text_test)))

vectorizer = CountVectorizer(max_df=0.90, min_df=0.01, max_features=5000)
vectorizer.fit(text_train)
X_train, y_train = vectorizer.transform(text_train), (label_train == 'pos').values
X_test, y_test = vectorizer.transform(text_test), (label_test == 'pos').values
pruebas = []


alphas = list(range(50, 101, 5))

for alpha in alphas:
    pca = PCA(alpha)
    print("Alpha {}".format(alpha))
    print("Entrenando PCA")

    #print(type(X_train))
    pca.fit(X_train.toarray())
    #print(type(X_train))
    X_pca_train = pca.transform(X_train)
    #print(type(X_train))
    X_pca_test = pca.transform(X_test)
    acc = []
    for k in range(100,151,3):
        ## Creo y entreno
        clf = KNNClassifier(k)
        clf.fit(X_pca_train, y_train)

        # Predigo
        y_pred = clf.predict(X_pca_test)

        # Me fijo el accuracy
        accuracy = accuracy_score(y_test, y_pred)
        acc.append(accuracy)
        print(acc)
        #print("k = {} alpha = {} ----> {}".format(k, alpha, accuracy))
        
        pruebas.append({
            "k": k,
            "alpha": alpha,
            "acc": acc,
        })
    plt.plot(list(range(100,151,3)), acc, '-o')
    print(acc)
        

plt.legend(alphas, loc='upper left')
plt.xlabel("Vecinos")
plt.ylabel("Accuracy")
plt.xticks(list(range(100,151,3)))
plt.title("Performance de clasificador en función a cantidad de vecinos y alpha");

Cantidad de instancias de entrenamiento = 9375
Cantidad de instancias de test = 3125


KeyboardInterrupt: 

In [ ]:
# TEST DE TAMAÑO DE INSTANCIAS DE ENTRENAMIENTO vs TEST CON KNN VARIABLE
import sentiment
res1 = []
division = list(range(9375,12500,3125)) #1/4, 1/2, 3/4 de la base se usan para train
rangos = [(x,12500-x) for x in division]
#Orig: 501
knns = list(range(80,151,5))

for n in division:
    print("Analizando n={}".format(n))
    acc = []
    
    #Divido train y test
    text_train = df[:n]["review"]
    label_train = df[:n]["label"]
    text_test = df[n+1:]["review"]
    label_test = df[n+1:]["label"]
    print("Cantidad de instancias de entrenamiento = {}".format(len(text_train)))
    print("Cantidad de instancias de test = {}".format(len(text_test)))
    #Hago BoW
    print("Vectorizando...")
    vectorizer = CountVectorizer(max_df=0.90, min_df=0.01, max_features=5000)
    vectorizer.fit(text_train)
    X_train, y_train = vectorizer.transform(text_train), (label_train == 'pos').values
    X_test, y_test = vectorizer.transform(text_test), (label_test == 'pos').values
    
    #Aplico KNN
    for k in knns:
        print("Para {} vecinos".format(k))        
        clf = sentiment.KNNClassifier(k)
        print("Hago fit...")
        clf.fit(X_train, y_train)
        #Predigo
        print("Predigo...")
        y_pred = clf.predict(X_test)
        #Calculo accuracy y guardo resultado
        acc.append(accuracy_score(y_test, y_pred))
        print("Accuracy: {}".format(acc))
    plt.plot(knns, acc, '-o')

plt.legend(rangos, loc='upper left')
plt.xlabel("Vecinos")
plt.ylabel("Accuracy")
plt.xticks(knns)
plt.title("Performance de clasificador en función de tamaño de la base de entrenamiento");

Analizando n=9375
Cantidad de instancias de entrenamiento = 9375
Cantidad de instancias de test = 3124
Vectorizando...


In [ ]:
vectorizer = CountVectorizer(max_df=0.90, min_df=0.01, max_features=5000)

vectorizer.fit(text_train)

X_train, y_train = vectorizer.transform(text_train), (label_train == 'pos').values
X_test, y_test = vectorizer.transform(text_test), (label_test == 'pos').values